In [2]:
import numpy as np
from scipy import signal, io
import matplotlib.pyplot as plt
from acquisition_tools import cal_matrix, estimate_rpm
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import ipywidgets as widgets
from IPython.display import display, Audio
from datetime import datetime
%matplotlib qt

plt.rcParams['text.usetex'] = False
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['font.size'] = 16


In [19]:
# Load the .mat file
# Open a file dialog to choose a .mat file
root = Tk()
# root.withdraw()  # Hide the main window
file_path = askopenfilename(initialdir='./results/Turbulent', filetypes=[("MAT files", "*.mat")])
root.destroy()  # Destroy the main window

exp_data = io.loadmat(file_path)
prop_name = file_path.split('/')[-1].split('_')[0]

In [20]:
time = exp_data['t']
fs_analog = exp_data['Fs_analog'].flatten()[0]
station_time = exp_data['station_time'].flatten()[0]

diameter_in = exp_data['D_inch']
pitch_in = exp_data['pitch']

diameter_m = diameter_in * 0.0254
pitch_m = pitch_in * 0.0254



voltage = exp_data['data']
force_net = exp_data['force_net']
force_raw = exp_data['force']
t_analog = np.linspace(0, len(force_raw)/fs_analog, len(force_raw)).flatten()

z_pulse = voltage[:, 6]
z_pulse = np.where(z_pulse > 1, 5, z_pulse)

mic_signal = voltage[:, 7]

rpm_sweep = exp_data['rpm_sweep'].flatten()
U_inf = exp_data['U_inf'].flatten()
rpm_command = exp_data['rpm_command'].flatten()
rpm_feedback = exp_data['rpm'].flatten()

pre_bias = exp_data['bias_mean'].flatten()
post_bias = exp_data['bias_mean_post'].flatten()

pre_bias_file = exp_data['bias_file'].flatten()[0]
post_bias_file = exp_data['bias_file_post'].flatten()[0]

pre_bias_timestamp = pre_bias_file.split('_')[-1].split('.')[0]
post_bias_timestamp = post_bias_file.split('_')[-1].split('.')[0]

pre_time = datetime.strptime(pre_bias_timestamp, "%H%M%S")
post_time = datetime.strptime(post_bias_timestamp, "%H%M%S")
time_difference = (post_time - pre_time).total_seconds()

rho = exp_data['rho']

In [23]:
mano_data = exp_data['data_mano']
u = ((mano_data[:, 1] * 200 * 2 / (rho))**0.5).flatten()
time_u = np.linspace(0, time.max(), len(u))

plt.figure(figsize=(10, 5))
plt.plot(time_u, u, label='Velocity (u)', color='blue')
plt.xlabel('Time [s]')
plt.ylabel('Velocity [m/s]')
plt.title('Velocity (u) as a function of Time')
plt.legend()
plt.grid()
plt.show()

In [24]:
indices = np.where(np.diff(z_pulse) > 1)[0]
rpm = 60/np.diff(t_analog.flatten()[indices])
t_rpm = t_analog.flatten()[indices[:-1]]

In [25]:
time_diffs = np.diff(time.flatten())
mean_fs = 1 / np.mean(time_diffs)
print(mean_fs)

20.003788528116008


In [26]:
plt.plot(t_rpm, rpm)

In [27]:
# Resample rpm to match fs_analog
num_samples = int(len(t_analog))
rpm_resampled = signal.resample(rpm, num_samples)
rpm_command_resampled = signal.resample(rpm_command, num_samples)
u_resampled = signal.resample(u, num_samples)

In [40]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, voltage[:, 0], "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("Ch1 [V]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, voltage[:, 1], "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("Ch2 [V]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, voltage[:, 2], "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("Ch3 [V]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, voltage[:, 3], "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("Ch4 [V]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, voltage[:, 4], "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("Ch5 [V]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, voltage[:, 5], "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("Ch6 [V]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

In [12]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, force_raw[:, 0], "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, force_raw[:, 1], "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, force_raw[:, 2], "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, force_raw[:, 3], "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, force_raw[:, 4], "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, force_raw[:, 5], "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

In [28]:
dt = 1/fs_analog
order = 2
cutoff = 5
nyq = 0.5 * fs_analog
normal_cutoff = cutoff / nyq

b, a = signal.butter(order, normal_cutoff)
z_l = z_d = signal.lfilter_zi(b, a)

Fz_raw = force_net[:, 2]
Tz_raw = force_net[:, 5]

Fx = signal.lfilter(b, a, force_net[:, 0], zi=z_l)[0]
Fy = signal.lfilter(b, a, force_net[:, 1], zi=z_l)[0]
Fz = signal.lfilter(b, a, force_net[:, 2], zi=z_l)[0]
Tx = signal.lfilter(b, a, force_net[:, 3], zi=z_l)[0]
Ty = signal.lfilter(b, a, force_net[:, 4], zi=z_l)[0]
Tz = signal.lfilter(b, a, force_net[:, 5], zi=z_l)[0]

In [42]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, Fx, "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, Fy, "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, Fz, "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, Tx, "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, Ty, "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, -Tz, "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

Windowing

In [29]:
window_size = station_time  # window size in seconds
samples_per_window = int(window_size * fs_analog)  # number of samples per window

# Partition data and t_analog into 40-second windows
t_analog_windows = [t_analog[i:i + samples_per_window] for i in range(0, len(t_analog), samples_per_window) if i + samples_per_window <= len(t_analog)]
Fz_windows = [Fz_raw[i:i + samples_per_window] for i in range(0, len(Fz_raw), samples_per_window) if i + samples_per_window <= len(Fz_raw)]
Tz_windows = [Tz_raw[i:i + samples_per_window] for i in range(0, len(Tz_raw), samples_per_window) if i + samples_per_window <= len(Tz_raw)]
rpm_windows = [rpm_resampled[i:i + samples_per_window] for i in range(0, len(rpm_resampled), samples_per_window) if i + samples_per_window <= len(rpm_resampled)]
u_windows = [u_resampled[i:i + samples_per_window] for i in range(0, len(u_resampled), samples_per_window) if i + samples_per_window <= len(u_resampled)]


# Calculate the mean of each window, leaving out the first 10 seconds and the last 2 seconds
mean_Fz_windows = np.array([np.mean(window[int(10 * fs_analog):-int(2 * fs_analog)]) for window in Fz_windows]).flatten()
mean_Tz_windows = np.array([np.mean(window[int(10 * fs_analog):-int(2 * fs_analog)]) for window in Tz_windows]).flatten()
mean_rpm_windows = np.array([np.mean(window[int(10 * fs_analog):-int(2 * fs_analog)]) for window in rpm_windows]).flatten()
mean_u_windows = np.array([np.mean(window[int(10 * fs_analog):-int(2 * fs_analog)]) for window in u_windows]).flatten()
mean_n_windows = np.array(mean_rpm_windows).flatten() / 60

std_Fz_windows = np.std(Fz_windows, axis=1)

# Print the mean values of each window
print(mean_rpm_windows/60)
print(mean_Fz_windows)
print(std_Fz_windows)
print(mean_rpm_windows)
# print(std_Tz_windows)
print(mean_u_windows)

[65.61905296 70.31310047]
[10.39947244 12.33526237]
[4.89691817 6.40890924]
[3937.14317776 4218.78602848]
[8.28922696 8.32884948]


In [30]:
J = mean_u_windows / (mean_n_windows * diameter_m)
CT = (mean_Fz_windows / (rho * mean_n_windows**2 * diameter_m**4))
CQ = (-np.array(mean_Tz_windows) / (rho * mean_n_windows**2 * diameter_m**5))
CP = CQ * 2 * np.pi
eta = J * CT / CP
mean_rpm_windows = mean_rpm_windows.flatten()

print(J)

[[0.31083533 0.29147081]]


In [31]:
# Create a figure and two subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# First subplot
ax1.plot(J, CT, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax1.grid()
ax1.set_xlabel('J')
ax1.set_ylabel('$C_T$')
ax1.set_xlim(0.1, .8)
ax1.set_ylim(0, 0.2)
ax1.set_xticks(np.arange(0.2, .8, 0.1))
# ax1.legend(loc = 'lower left')

# Second subplot
ax2.plot(J, CP, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax2.set_xlim(0.1, .8)
ax2.set_ylim(0., 0.2)
ax2.set_xlabel('J')
ax2.set_ylabel('$C_P$')
ax2.set_xticks(np.arange(0.2, .8, 0.1))
ax2.grid()

# Third subplot
ax3.plot(J, eta, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax3.set_xlim(0.1, .8)
ax3.set_ylim(0, 1)
ax3.set_xlabel('J')
ax3.set_ylabel('$\eta$')
ax3.set_xticks(np.arange(0.2, .8, 0.1))
ax3.grid()

# Show the plot
plt.tight_layout()
plt.show()

<>:28: SyntaxWarning: invalid escape sequence '\e'
<>:28: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Lab7\AppData\Local\Temp\ipykernel_4376\4293431116.py:28: SyntaxWarning: invalid escape sequence '\e'
  ax3.set_ylabel('$\eta$')


In [106]:
# Create a figure and two subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# First subplot
ax1.plot(mean_rpm_windows/60, CT.T, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax1.grid()
ax1.set_xlabel("BPF/N [Hz]")
ax1.set_ylabel('$C_T$')
ax1.set_xlim(150, 0)
ax1.set_ylim(0, 0.2)
# ax1.invert_xaxis()
# ax1.set_xticks(np.arange(0.2, .8, 0.1))
# ax1.legend(loc = 'lower left')

# Second subplot
ax2.plot(mean_rpm_windows/60, CP.T, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax2.set_xlim(150, 0)
ax2.set_ylim(0., 0.2)
ax2.set_xlabel("BPF/N [Hz]")
ax2.set_ylabel('$C_P$')
# ax2.set_xticks(np.arange(0.2, .8, 0.1))s
ax2.grid()
# ax2.invert_xaxis()

# Third subplot
ax3.plot(mean_rpm_windows/60, eta.T, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax3.set_ylim(0, 1)
ax3.set_xlim(150, 0)
ax3.set_xlabel("BPF/N [Hz]")
ax3.set_ylabel('$\eta$')
# ax3.set_xticks(np.arange(0.2, .8, 0.1))s
ax3.grid()
# ax3.invert_xaxis()

# Show the plot
plt.tight_layout()
plt.show()

<>:30: SyntaxWarning: invalid escape sequence '\e'
<>:30: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Lab7\AppData\Local\Temp\ipykernel_17208\1131243699.py:30: SyntaxWarning: invalid escape sequence '\e'
  ax3.set_ylabel('$\eta$')


In [17]:
# Define the window size in samples for 1 second
window_size_samples = 30 * int(fs_analog)  # fs_analog is the sampling frequency

# Partition Fz into 1-second windows
Fz_1s_windows = np.array([Fz_raw[i:i + window_size_samples] for i in range(0, len(Fz), window_size_samples) if i + window_size_samples <= len(Fz)])

# Calculate the standard deviation for each 1-second window
std_Fz_1s_windows = np.array([np.std(window) for window in Fz_1s_windows])

# Create a time array for the center of each 1-second window
time_1s_centers = np.array([np.mean(window) for window in [t_analog[i:i + window_size_samples] for i in range(0, len(t_analog), window_size_samples) if i + window_size_samples <= len(t_analog)]])

plt.plot(time_1s_centers, std_Fz_1s_windows, ".-")

In [18]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# First subplot
ax1.plot(t_analog, Fz, "-", label='Thrust', linewidth = 2, color='k')
ax1.set_ylim(-20, 40)

for i, window in enumerate(Fz_windows):
    start_time = t_analog_windows[i][int(10 * fs_analog)]
    end_time = t_analog_windows[i][-1]
    ax1.axvspan(start_time, end_time, color='gray', alpha=0.3)
    ax2.axvspan(start_time, end_time, color='gray', alpha=0.3)

ax1_twiny = ax1.twinx()
ax1_twiny.plot(t_rpm, rpm, "-", label='RPM', linewidth=2, color='blue')
ax1_twiny.set_ylabel("RPM")
ax1_twiny.set_ylim(0, 6000)

ax1_twiny.plot(t_analog, rpm_command_resampled, "-", label='RPM Command', linewidth=1, color='red')
ax1_twiny.set_ylabel("RPM Command")

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twiny.get_legend_handles_labels()
ax1_twiny.legend(lines + lines2, labels + labels2, loc='lower left')

# # Add error bars for each 1-second center
# ax1.errorbar(time_1s_centers, np.mean(Fz_1s_windows, axis=1), yerr=std_Fz_1s_windows, fmt='o', color='green', label='1s Mean with Std Dev')


ax1.set_xlabel("Time [s]")
ax1.set_ylabel("T [N]")
ax1.grid()

# Second subplot
ax2.plot(t_analog, -Tz, "-", label='Torque', linewidth = 2, color='red')
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Q [N.m]")
ax2.set_ylim(-0.5, 1)
ax2.legend()
ax2.grid()

plt.tight_layout()
plt.show()

C:\Users\Lab7\AppData\Local\Temp\ipykernel_4376\2405366462.py:41: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()


m:\Efe\TurboProp\venv\Lib\site-packages\ipykernel\eventloops.py:145: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  el.exec() if hasattr(el, "exec") else el.exec_()


Zero Drift Analysis

In [20]:
print(pre_bias)
print(post_bias)
print((pre_bias - post_bias))
print(100*(pre_bias - post_bias)/pre_bias, "%")

[-3.18978061 -5.30099563 -4.07992375 -0.17473666 -0.51163617  0.05500724]
[-3.17634994 -5.31257229 -4.09941266 -0.17484322 -0.51305984  0.05563058]
[-0.01343067  0.01157666  0.01948891  0.00010656  0.00142368 -0.00062334]
[ 0.4210531  -0.21838661 -0.47767829 -0.06098308 -0.27826001 -1.1332031 ] %


In [21]:
# Create a time array for interpolation
interpolation_time = np.linspace(0, time_difference, int(time_difference * fs_analog))

# Interpolate between pre_bias and post_bias
bias_interpolated = np.array([
    np.interp(interpolation_time, [0, time_difference], [pre_bias[i], post_bias[i]])
    for i in range(len(pre_bias))
])

# Subtract bias_interpolated from force_raw
force_net_interpolated = force_raw - bias_interpolated.T[0:len(force_raw)]

In [106]:
Fz_raw_interpolated = force_net_interpolated[:, 2]
Tz_raw_interpolated = force_net_interpolated[:, 5]

Fx_interpolated = signal.lfilter(b, a, force_net_interpolated[:, 0], zi=z_l)[0]
Fy_interpolated = signal.lfilter(b, a, force_net_interpolated[:, 1], zi=z_l)[0]
Fz_interpolated = signal.lfilter(b, a, force_net_interpolated[:, 2], zi=z_l)[0]
Tx_interpolated = signal.lfilter(b, a, force_net_interpolated[:, 3], zi=z_l)[0]
Ty_interpolated = signal.lfilter(b, a, force_net_interpolated[:, 4], zi=z_l)[0]
Tz_interpolated = signal.lfilter(b, a, force_net_interpolated[:, 5], zi=z_l)[0]

In [107]:
# Partition data and t_analog into 40-second windows
Fz_interpolated_windows = [Fz_raw_interpolated[i:i + samples_per_window] for i in range(0, len(Fz_raw), samples_per_window) if i + samples_per_window <= len(Fz_raw)]
Tz_interpolated_windows = [Tz_raw_interpolated[i:i + samples_per_window] for i in range(0, len(Tz_raw), samples_per_window) if i + samples_per_window <= len(Tz_raw)]


# Calculate the mean of each window, leaving out the first 10 seconds and the last 2 seconds
mean_Fz_interpolated_windows = np.array([np.mean(window[int(10 * fs_analog):-int(2 * fs_analog)]) for window in Fz_interpolated_windows])
mean_Tz_interpolated_windows = np.array([np.mean(window[int(10 * fs_analog):-int(2 * fs_analog)]) for window in Tz_interpolated_windows])

std_Fz_interpolated_windows = np.std(mean_Fz_interpolated_windows)

In [108]:
CT_interpolated = mean_Fz_interpolated_windows / (rho * mean_n_windows**2 * diameter_m**4)
CQ_interpolated = -np.array(mean_Tz_interpolated_windows) / (rho * mean_n_windows**2 * diameter_m**5)
CP_interpolated = CQ_interpolated * 2 * np.pi
eta_interpolated = J * CT_interpolated / CP_interpolated

In [113]:
# Create a figure and two subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# First subplot
ax1.plot(J, CT_interpolated, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax1.grid()
ax1.set_xlabel('J')
ax1.set_ylabel('$C_T$')
ax1.set_xlim(0.2, .8)
ax1.set_ylim(0, 0.2)
ax1.set_xticks(np.arange(0.2, .8, 0.1))
# ax1.legend(loc = 'lower left')

# Second subplot
ax2.plot(J, CP_interpolated, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax2.set_xlim(0.2, .8)
ax2.set_ylim(0., 0.2)
ax2.set_xlabel('J')
ax2.set_ylabel('$C_P$')
ax2.set_xticks(np.arange(0.2, .8, 0.1))
ax2.grid()

# Third subplot
ax3.plot(J, eta_interpolated, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax3.set_xlim(0.2, .8)
ax3.set_ylim(0, 1)
ax3.set_xlabel('J')
ax3.set_ylabel('$\eta$')
ax3.set_xticks(np.arange(0.2, .8, 0.1))
ax3.grid()

# Show the plot
plt.tight_layout()
plt.show()

<>:28: SyntaxWarning: invalid escape sequence '\e'
<>:28: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Lab7\AppData\Local\Temp\ipykernel_16392\3179712500.py:28: SyntaxWarning: invalid escape sequence '\e'
  ax3.set_ylabel('$\eta$')


Audio Signal

In [217]:
# Create an audio widget
audio_widget = Audio(mic_signal, rate=fs_analog, autoplay=False)

# Display the audio player and the slider
display(audio_widget)

In [322]:
plt.figure(figsize=(10, 5))
plt.specgram(mic_signal, Fs=fs_analog, NFFT=4096*2, noverlap=128, cmap='viridis')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.title('Spectrogram of Microphone Signal')
plt.colorbar(label='Intensity [dB]')
plt.show()

In [432]:
# Compute the FFT of force_net[:, 2]
force_net_fft = np.fft.fft(force_net[:, 2])
frequencies = np.fft.fftfreq(force_net.shape[0], d=1/fs_analog)

# Make the FFT one-sided
force_net_fft_one_sided = force_net_fft[:len(force_net_fft)//2]
frequencies_one_sided = frequencies[:len(frequencies)//2]

# Plot the one-sided FFT of force_net[:, 2]
plt.figure(figsize=(12, 8))
plt.plot(frequencies_one_sided, np.abs(force_net_fft_one_sided))
plt.xlabel('Frequency [Hz]')
plt.ylabel('|FFT(Force Z)|')
plt.title('One-Sided FFT of Force Z')
plt.grid()
plt.tight_layout()
plt.show()

In [326]:
# Perform PSD on Fz
f, Pxx_den = signal.welch(force_net[:, 2], fs_analog, nperseg=1024)

# Plot the PSD
plt.figure(figsize=(10, 6))
plt.semilogy(f, Pxx_den)
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.title('Power Spectral Density of Fz')
plt.grid()
plt.show()

In [115]:
# Define the variables to save

data_to_save = {
    'mic_signal': mic_signal,
    'fs_analog': fs_analog,
    'rpm_resampled': rpm_resampled
}

# Save the variables to a .mat file
io.savemat('7086rpm_no_flow.mat', data_to_save)